In [80]:
import pandas as pd
import numpy as np
import unicodedata
import matplotlib.pyplot as plt
import plotly.express as px
import os # to check if directory exists and create it if it doesn't
from datetime import datetime # to parse speech date
import spacy
import en_core_web_md
import csv
import nltk
import re
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn import manifold # for t-sne
import plotly.graph_objects as go
import kaleido

# [To PCA on tidy data csv](#pcatidydata)

In [2]:
# Load the two data sets needed
# string encodings to try utf_8, iso8859_15
facetPlotData = pd.read_csv('facetPlotData.csv')
df = pd.read_csv('fullEmotionData.csv', encoding='utf_8') # apostrophes aren't being read correctly
# Pandas reads date as string, cast to datetime object
df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')
facetPlotData['date'] = pd.to_datetime(facetPlotData['date'], format='%Y-%m-%d')
facetPlotData.shape # Should be (3000, 4) 

(3000, 4)

In [3]:
sources = ['text_oba', 'text_nyt', 'text_wsj']
for source in sources:
    tmp1 = [re.sub(r"[^\w\s^']", " ", text).lower() for text in df[source]]
    df[source] = [re.sub(' +', ' ', text).replace('\n', '').replace("â " , "'") for text in tmp1]

In [4]:
# Read in new articles from OtherSources directory
# Load up the files
path = './OtherSources/' 
list_of_files = []

# Get file names in target directory
for root, dirs, files in os.walk(path):
    for file in files:
        if file.endswith('.txt'):
            list_of_files.append(os.path.join(root,file))

# Open files and read in text
speeches = []
for file in list_of_files:
    with open(file, encoding='utf-8') as f:
        text = f.read()
    f.close()
    speeches.append(text)

#clean out goofy unicode  space characters 
speeches = [unicodedata.normalize("NFKD", speech) for speech in speeches]

info = [(file.split('_')[0][15:]+'.txt', file.split('_')[1][:-4]) for file in list_of_files]

# speeches[i] has the article text
# info[i][0] has the filename to match row of original Obama speech
# info[i][1] has the source name for naming the column
# Some new source/column names are: FoxNews, InfoWars, NationalReview, Intercept

for i in range(len(info)):
    df[info[i][1]] = None

for i in range(len(list_of_files)):
    df.loc[df['file'] == info[i][0], [info[i][1]]] = speeches[i]

<A HREf="https://arxiv.org/abs/1905.05583">Paper with some stuff on dealing with long text</A> This doesn't seem to be an issue with spacy encodings

In [5]:
# Clean up Obama speeches and what the heck, let's do NYT and WSJ too, just to be sure no more unicode garbage
df['text_oba'] = [unicodedata.normalize("NFKD", speech) for speech in df['text_oba']]
df['text_nyt'] = [unicodedata.normalize("NFKD", speech) for speech in df['text_nyt']]
df['text_wsj'] = [unicodedata.normalize("NFKD", speech) for speech in df['text_wsj']]

In [6]:
nlp = spacy.load('en_core_web_md')

In [7]:
# This takes a little bit to calculate the encodings... maybe a minute
# Encode all the texts and save to source specific columns
sources = ['text_oba', 'text_nyt', 'text_wsj', 'FoxNews', 'InfoWars', 'NationalReview', 'Intercept']
enc_col = ['enc_oba', 'enc_nyt', 'enc_wsj', 'enc_fn', 'enc_iw', 'enc_nr', 'enc_int']

# initialize the new encoding columns
for col in enc_col:
    df[col]=None

for i in range(len(df)):
    for j in range(len(sources)):
        #print(i, sources[j])
        if df[sources[j]][i] != None:
            # dfmi.loc[:, ('one', 'second')]
            df[enc_col[j]][i] = list(nlp(df[sources[j]][i]).vector)

/tmp/ipykernel_36646/3038641089.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[enc_col[j]][i] = list(nlp(df[sources[j]][i]).vector)


In [8]:
pcaBiplotData = pd.DataFrame(columns=['date','source'])

In [9]:
# Reshape data for pca analysis and plotting
for i in range(len(df)):
    for j in range(len(sources)):
        if df.loc[i,sources[j]] != None:
            index = (i*len(sources))+j
            pcaBiplotData.loc[index, 'date' ] = pd.to_datetime(df['date'][i])
            pcaBiplotData.loc[index, 'source' ] = sources[j]
            for k in range(len(df['enc_oba'][99])):
                columnname = 'enc'+str(k+1)
                # initialize column, not sure this is necessary
                pcaBiplotData.loc[index, columnname] = None
                encoding = enc_col[j]
                pcaBiplotData.loc[index, columnname] = df[encoding][i][k]
pcaBiplotData = pcaBiplotData.reset_index()

/tmp/ipykernel_36646/1484095954.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  pcaBiplotData.loc[index, columnname] = None
/tmp/ipykernel_36646/1484095954.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  pcaBiplotData.loc[index, columnname] = None
/tmp/ipykernel_36646/1484095954.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragment

In [38]:
pcaBiplotData['date'] = pd.to_datetime(pcaBiplotData['date'], format='%Y-%m-%d')

In [39]:
# Selecting dataframe subset with columns that start with 'enc'
# pcaBiplotData.loc[:, pcaBiplotData.columns.str.startswith('enc')]

In [40]:
# All encoding values are already on same scale
pca = PCA(n_components = 2)
pcafit = pca.fit_transform(pcaBiplotData.loc[:, pcaBiplotData.columns.str.startswith('enc')])

In [41]:
plot_data = pd.DataFrame()
plot_data = pd.concat([pd.DataFrame(pcafit), pcaBiplotData['date'], pcaBiplotData['source']], axis=1, ignore_index=True)

<A HREF="https://plotly.com/python/discrete-color/#color-sequences-in-plotly-express">Plotly discrete color palettes/sequences</A><BR><A HREF="https://plotly.com/python/plotly-express/">Plotly Express docs</A>

In [42]:
fig = px.scatter(pcafit, 
                 x=0, 
                 y=1, 
                 color=pcaBiplotData['source'],
                 hover_name=pcaBiplotData['date'],
                 color_discrete_sequence=px.colors.qualitative.Bold)
fig.show()

In [14]:
# What is that NYT way out there?
plot_data.loc[plot_data[0].idxmax()]

0              10.995829
1               3.820567
2    2012-07-21 00:00:00
3               text_oba
Name: 140, dtype: object

Very short article announcing new Intel fab and mentioning that Obama spoke at Intel.

### Save the data

In [15]:
#pcaBiplotData.to_csv('pcaBiplotData.csv', index=False)

In [16]:
#df.to_csv('df_encodings.csv')

Playing with python's PCA

In [17]:
pca2 = PCA(n_components = 10)
pcafit2 = pca2.fit_transform(pcaBiplotData.loc[:, pcaBiplotData.columns.str.startswith('enc')])

In [18]:
# The singular values corresponding to each of the selected components - guessing they're refering to eigenvalues
print(pca2.singular_values_)

[81.78093018 42.05430485 31.00536398 29.35742016 20.7621332  17.90038185
 17.76776161 16.05083271 13.82075759 13.030196  ]


In [19]:
# The amount of variance explained by each of the selected components.
print(pca2.explained_variance_)

[21.64440304  5.72350989  3.11110872  2.78918485  1.39503616  1.03696981
  1.02166134  0.83375156  0.61816615  0.54946928]


In [20]:
# Percentage of variance explained by each of the selected components.
print(pca2.explained_variance_ratio_)

[0.47480208 0.12555368 0.06824678 0.06118491 0.03060219 0.02274747
 0.02241166 0.01828958 0.01356039 0.01205342]


In [21]:
# Sklearn's T-SNE
# https://scikit-learn.org/stable/modules/generated/sklearn.manifold.TSNE.html
from sklearn import manifold

In [44]:
# Perplexity value can greatly influence the shape of the results, typical ranges from 5 to 50
# Some perplexity values with decent separation: 6,7,9,10,12,14,15
t_sne = manifold.TSNE(
    n_components=2,
    perplexity=10,
    init="random",
    n_iter=250,
    random_state=0,
);
tsne_data = pcaBiplotData.loc[:, pcaBiplotData.columns.str.startswith('enc')]
S_t_sne = t_sne.fit_transform(tsne_data);
fig = px.scatter(S_t_sne, 
                 x=0, 
                 y=1, 
                 color=pcaBiplotData['source'],
                 hover_name=pcaBiplotData['date'],
                 color_discrete_sequence=px.colors.qualitative.Bold)
fig.show()

/home/muddy/.local/lib/python3.10/site-packages/sklearn/manifold/_t_sne.py:810: FutureWarning:

The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.



In [23]:
# Join PCA components to biplot dataframe and then save
pcaBiplotData.drop(['index'], axis=1, inplace=True)
pcafitdf = pd.DataFrame(pcafit, columns=['PCA1', 'PCA2'])
pcaBiplotData = pcaBiplotData.join(pcafitdf)

In [24]:
#pcaBiplotData.to_csv('pcaBiplotData.csv', index=False)

### <a id='pcatidydata'>Try PCA on the big tidy data set with emo, pos, sentence and word counts</a>

In [3]:
scaling=StandardScaler()
df_tidy = pd.read_csv('tidy_data.csv')
#df_tidy.columns

In [39]:
pca_data = df_tidy.drop(['date', 'source', 'enc_value','PCA1', 'PCA2','num_sents', 'num_words'], axis=1, inplace=False)
scaling.fit(pca_data)
scaled_data = scaling.transform(pca_data)
pca = PCA(n_components = 2)
pcafit2 = pca.fit_transform(scaled_data)

In [40]:
fig = px.scatter(pcafit2, 
                 x=0, 
                 y=1, 
                 color=df_tidy['source'],
                 color_discrete_sequence=px.colors.qualitative.Bold)
fig.show()

In [41]:
print(pca.explained_variance_ratio_)

[0.33908322 0.16506299]


In [42]:
print(pca.singular_values_)

[71.31793855 49.75886645]


In [53]:
loadings=pd.DataFrame(pca.components_.T, columns=['PC1', 'PC2'], index=pca_data.columns)
def large_bold(val):
    bold = 'bold' if abs(val) > 0.18 else ''
    return 'font-weight: %s' % bold
loadings.T.style.applymap(large_bold) # .T makes it wide instead of tall

,ADJ,ADP,ADV,AUX,CCONJ,DET,INTJ,NOUN,NUM,PART,PRON,PROPN,PUNCT,SCONJ,SYM,VERB,anger,anticipation,disgust,fear,joy,negative,positive,sadness,surprise,trust,num_unique_words,depth,TBsubjectivity,TBpolarity,words_per_sentence,char_count,syl_count,word_count,char_per_word,syl_per_word,sent_count,word_per_sent,dc_word_count,gf_word_count,poly_word_count,dc_word_perc,gf_word_perc,poly_word_perc,ari,flesch_kincaid,coleman_liau,dale_chall,gunning_fog,smog
PC1,-0.167454,-0.169442,-0.025360,-0.146019,-0.115724,-0.171191,0.161070,-0.175604,-0.007077,-0.152370,-0.057085,-0.136704,-0.171323,-0.018019,0.076269,-0.171430,-0.043376,0.058347,-0.000095,-0.060229,0.118699,-0.051894,0.008079,0.003931,0.030234,-0.004633,0.173243,-0.175198,0.104792,0.115483,-0.157923,0.174021,0.172315,0.181860,-0.182341,-0.179789,0.195367,-0.151936,0.162632,0.142131,0.141894,-0.171388,-0.166694,-0.171637,-0.184010,-0.184644,-0.189192,-0.179811,-0.178789,-0.183604
PC2,0.172198,0.187543,0.033855,0.185056,0.114378,0.187895,-0.056674,0.183391,-0.114525,0.162208,0.047754,0.127821,0.188561,0.025831,-0.158616,0.193874,-0.002320,0.000157,0.009148,0.010402,0.050616,-0.025371,-0.011454,0.010641,0.019828,-0.024384,-0.134787,-0.158762,0.088322,0.031032,-0.190637,-0.130198,-0.133203,-0.118264,-0.160269,-0.171813,-0.081596,-0.195582,-0.147714,-0.171997,-0.170964,-0.130252,-0.183632,-0.182047,-0.207665,-0.210552,-0.174319,-0.202604,-0.215686,-0.213905


In [60]:
fig = px.bar(loadings, 
             x=loadings.index, 
             y='PC1', 
             color='PC1', 
             title = 'PC1 loadings')
fig.show()

In [61]:
fig = px.bar(loadings, 
             x=loadings.index, 
             y='PC2', 
             color='PC2', 
             title = 'PC2 loadings')
fig.show()

<A HREF="https://plotly.com/python/figure-factory-table/">Plotly Figure Factory</A>

In [81]:
to_png = loadings.copy()
to_png['var'] = to_png.index
table = ff.create_table(round(to_png,3))
table.update_layout(
    autosize=False,
    width=400,
    height=700)

In [ ]:
# Not working
import kaleido
table.write_image("table_plotly.png", scale=2)

In [106]:
font_color=['black']*2+[['red' if boolv else 'black' for boolv in abs(to_png['PC1'])>0.16]]
table_trace = go.Table(
                 header = dict(height = 50,
                               values = [['Var'], ['PC1'], ['PC2']],
                               align = ['center']*3,
                               fill_color = '#686dea',
                               font_color = '#ffffff',
                               font_size =20),
                 cells = dict(values = [to_png['var'], round(to_png['PC1'],3), round(to_png['PC2'],3)],
                              line = dict(color='#506784'),
                              align = ['left']*3,
                              font_color=font_color,
                              font_family="Arial", 
                              font_size=14,   
                              height = 25,
                              fill = dict(color='rgb(245,245,245)'))
                             )
layout = go.Layout(width=600, height=800, autosize=False,
                   title_text='Principle Components',
                   title_x=0.5, showlegend=False)
fig = go.Figure(data=[table_trace], layout=layout)
fig.show()

In [112]:
fig = px.imshow(to_png,
                text_auto=True
               )
fig.update_xaxes(side="top")
fig.show()

<A HREF="https://distill.pub/2016/misread-tsne/">Some info about t-sne</A>

In [6]:
# Maybe the right perplexity in t-sne will separate these .... mostly not
# Perplexity value can greatly influence the shape of the results, typical ranges from 5 to 50
# Some perplexity values with decent separation: 6,7,9,10,12,14,15
t_sne = manifold.TSNE(
    n_components=2,
    perplexity=43,
    init="random",
    n_iter=1200,
    random_state=0,
);
tsne_data = scaled_data
S_t_sne = t_sne.fit_transform(tsne_data);
fig = px.scatter(S_t_sne, 
                 x=0, 
                 y=1, 
                 color=df_tidy['source'],
                 color_discrete_sequence=px.colors.qualitative.Bold)
fig.show()

/home/muddy/.local/lib/python3.10/site-packages/sklearn/manifold/_t_sne.py:810: FutureWarning:

The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.



In [7]:
# ok then, how about MDS?
# https://scikit-learn.org/stable/modules/manifold.html#multidimensional-scaling
# Data needs to be a similarity or disimilarity matrix ... maybe another day
md_scaling = manifold.MDS(
    n_components=2,
    max_iter=50,
    n_init=4,
    random_state=0
)
S_scaling = md_scaling.fit_transform(scaled_data)

fig = px.scatter(S_scaling, 
                 x=0, 
                 y=1, 
                 color=df_tidy['source'],
                 color_discrete_sequence=px.colors.qualitative.Bold)
fig.show()